<h2>Data Analysis - Batch Processing - Quantification of cell populations</h2>

The following notebook is able to process the .csv files resulting from Batch Processing (Average Intensity or Colocalization) and:

1. Define cell populations based on single or multiple markers (positive, negative or a combination of both)
2. Plot resulting data using Plotly.
3. Extract numbers of cells positive for a marker based on colocalization (using a user-defined threshold).
4. Aggregate all per labels results in a single .csv file ("BP_populations_marker_+_summary_{method}.csv")
4. Save summary % results on a cell population basis in .csv file ("BP_populations_marker_+_summary_{method}.csv").

In [7]:
from pathlib import Path
from utils_data_analysis import calculate_perc_pops, plot_perc_pop_per_filename_roi

In [8]:
# Define the path containing your results
results_path = Path("./results/test_data/3D/MEC0.1")

# Input the method used to define cells as positive for a marker ("avg_int", "coloc") #TODO: "pixel_class"
method = "avg_int"

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
markers = [("ki67", 0, "nucleus"), ("neun", 1, "nucleus"), ("calbindin", 2, "cytoplasm")]

# Define the min_max average intensity parameters to select your populations of interest
# You have the possibility to define populations for the same marker (i.e. neun high and neun low)
# max_values are set to 255 since the test input images are 8-bit, higher bit depths can result in higher max avg_int values
min_max_per_marker = [
    {"marker": "ki67", "min_max": (110,255), "population":"ki67"},
    {"marker": "neun", "min_max": (20,80), "population":"neun_low"},
    {"marker": "neun", "min_max": (80,255), "population":"neun_high"},
    {"marker": "calbindin", "min_max": (10,255), "population":"calbindin"},]

# Define cell populations based on multiple markers (i.e. double marker positive (True) or marker positive (True) and marker2 negative (False))
# Based on populations in min_max_per_marker in case multiple pops per marker are defined, as in the case of "neun"
# For cell_pop defined by a single populations marker add a + so it does not have the same name as population in min_max_per_marker
cell_populations = [
    {"cell_pop": "neun_high+", "subpopulations": [("neun_high", True)]},
    {"cell_pop": "neun_low+", "subpopulations": [("neun_low", True)]},
    {"cell_pop": "non_prolif", "subpopulations": [("ki67", False)]},
    {"cell_pop": "prolif_neun_high", "subpopulations": [("neun_high", True), ("ki67", True)]},
    {"cell_pop": "prolif_neun_low", "subpopulations": [("neun_low", True), ("ki67", True)]},
    {"cell_pop": "non_prolif_neun_high", "subpopulations": [("neun_high", True), ("ki67", False)]},
    {"cell_pop": "non_prolif_neun_low", "subpopulations": [("neun_low", True), ("ki67", False)]},
    {"cell_pop": "neun_high_+_calbindin_+", "subpopulations": [("neun_high", True), ("calbindin", True)]},
    {"cell_pop": "neun_low_+_calbindin_+", "subpopulations": [("neun_low", True), ("calbindin", True)]},]

In [9]:
# Extract model and segmentation type from results Path
# Calculate percentages of each cell population, save them as a summary .csv
percentage_true, model_name, segmentation_type = calculate_perc_pops(results_path, method, min_max_per_marker, cell_populations)

percentage_true

,filename,ROI,ki67,neun_low,neun_high,calbindin,neun_high+,neun_low+,non_prolif,prolif_neun_high,prolif_neun_low,non_prolif_neun_high,non_prolif_neun_low,neun_high_+_calbindin_+,neun_low_+_calbindin_+
0,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,CA,1.740947,26.532033,61.072423,65.738162,61.072423,26.532033,98.259053,0.557103,0.696379,60.515320,25.835655,50.557103,12.743733
1,HI 1 Contralateral Mouse 8 slide 6 Neun Red ...,DG,5.370722,65.874525,16.872624,28.136882,16.872624,65.874525,94.629278,0.190114,3.089354,16.682510,62.785171,15.351711,11.834601
2,HI 1 Ipsilateral Mouse 8 slide 6 Neun Red Ca...,CA,5.146799,35.709232,2.193138,36.045278,2.193138,35.709232,94.853201,0.053060,0.565971,2.140078,35.143261,1.892466,14.503007
3,HI 1 Ipsilateral Mouse 8 slide 6 Neun Red Ca...,DG,5.146799,35.709232,2.193138,36.045278,2.193138,35.709232,94.853201,0.053060,0.565971,2.140078,35.143261,1.892466,14.503007


In [10]:
# Plot the resulting cell population percentages of a per filename per ROI basis
plot_perc_pop_per_filename_roi(percentage_true, model_name, segmentation_type)